## The output file parser

The parser for the output file is used in the same way as the input file (refer to tutorial 1 for the different ways to provide files to the parser).
However, it is much more configurable as the output file contains a lot of information that is not always useful. But the version support is not quite as extensive as the input parser. The output parser can parse out files starting form the MaX4 release of fleur

In [ ]:
from masci_tools.io.parsers.fleur import outxml_parser
outxml_parser?

In [ ]:
from pprint import pprint
parser_info = {'parser_warnings': []}
output_dict = outxml_parser('./files/SiLO_out.xml', parser_info_out=parser_info)

Let us look at the `parser_info` output first:

In [ ]:
pprint(parser_info)

We see the same version information and warnings as for the input parser here. Additionally we get information about the kind of calculation that was performed for the given input file. So in this case, we have a stock standard non-magnetic calculation. These modes determine how the output file is parsed. With these modes we obtain the following output:

In [ ]:
pprint(output_dict)

The structure of the output as seen here is based on the old aiida-fleur parser. For a output file with different calculation modes the output dictionary is automatically extended/modified with output related to the calculation modes.

In [ ]:
from pprint import pprint
parser_info = {'parser_warnings': []}
output_dict = outxml_parser('./files/Fe_bct_SOC_out.xml', parser_info_out=parser_info)
pprint(output_dict)
pprint(parser_info)

The `outxml_parser` allows the user to tailor the output to their needs. As a first step the `iteration_to_parse` argument allows the choice of which iteration should be parsed. By default this is set to `last`, which parses only the last valid (if there was an error the last complete one is parsed) iteration.

In [ ]:
#Parsing all iterations
from pprint import pprint
output_dict = outxml_parser('./files/Fe_bct_SOC_out.xml', iteration_to_parse='all')
pprint(output_dict)

In [ ]:
#Parsing the first iteration
from pprint import pprint
output_dict = outxml_parser('./files/Fe_bct_SOC_out.xml', iteration_to_parse='first')
pprint(output_dict)

In [ ]:
#Parsing the sixth iteration for example (indexing starts at 0) (any non negative integer is accepted if there were enough iterations)
from pprint import pprint
output_dict = outxml_parser('./files/Fe_bct_SOC_out.xml', iteration_to_parse=5)
pprint(output_dict)

You might see that the output for all iterations can be a bit overwhelming. If you are interested only in the most essential information setting the argument `minimal_mode` to True migth be interesting for you.

In [ ]:
#Parsing all iterations
from pprint import pprint
output_dict = outxml_parser('./files/Fe_bct_SOC_out.xml', iteration_to_parse='all', minimal_mode=True)
pprint(output_dict)

Here only the distances and total energies are parsed.

The flexible design of the output parser allows the user to do a lot of customization, without needing to modify the outxml_parser itself.
As an example let's say in addition to the parsed information we want to get the energy parameter output from the out.xml. We can define a task ourself in the form of a dictionary. 

In this example we define a task called `energy_parameters`, which has two parts. In `atomic_energy_parameters` we parse all attributes of the tags with the name `atomicEP` and put them into the output dictionary under a subdictionary with the name `atomic_energy_parameters`. If the path is unambiguously specified the outxml_parser will find the correct location automatically. Then we repeat the same procedure for the `loAtomicEP` tags. 

A detailed information about the keywords and arguments in this dictionary can be found in the documentation of `masci-tools`.

In [ ]:
from pprint import pprint
energy_parameter_tasks = {'energy_parameters': {'atomic_energy_parameters': {'parse_type': 'allAttribs',
                                                                             'path_spec': {'name': 'atomicEP'},
                                                                             'flat': False},
                                                'lo_energy_parameters': {'parse_type': 'allAttribs',
                                                                             'path_spec': {'name': 'loAtomicEP'},
                                                                        'flat': False}}}
output_dict = outxml_parser('./files/SiLO_out.xml', additional_tasks=energy_parameter_tasks)
pprint(output_dict)

We can even do custom conversions, which go beyond the normal parsing for our custom tasks. For this we need to define a function decorated with the `conversion_function` decorator and attach this function to our custom task by inserting a `_conversions` key with the name of the function in a list. As an example we calculate the mean energy parameter for atomic and local orbital energy parameters

In [ ]:
from pprint import pprint
from masci_tools.util.parse_tasks_decorators import conversion_function
import numpy as np

@conversion_function
def mean_energy_parameter(out_dict, parser_info_out=None): #This is the required function signature for a conversion function.
    
    eparas = out_dict['atomic_energy_parameters'].get('value')
    lo_eparas = out_dict['lo_energy_parameters'].get('value')
    
    out_dict['mean_atomic_ep'] = np.mean(eparas)
    out_dict['mean_lo_ep'] = np.mean(lo_eparas)
    
    return out_dict

energy_parameter_tasks = {'energy_parameters': {'_conversions': ['mean_energy_parameter'],
                                                'atomic_energy_parameters': {'parse_type': 'allAttribs',
                                                                             'path_spec': {'name': 'atomicEP'},
                                                                             'flat': False},
                                                'lo_energy_parameters': {'parse_type': 'allAttribs',
                                                                             'path_spec': {'name': 'loAtomicEP'},
                                                                        'flat': False}}}
output_dict = outxml_parser('./files/SiLO_out.xml', additional_tasks=energy_parameter_tasks)
pprint(output_dict)